# Continuing simulations
This notebook will show you:
- how to continue an existing ANN assisted TPS simulation from storage

Note: This notebook depends on files created in the first notebook `1_Toy_pytorch_simple_setup.ipynb`. Please do it first if you have not already.

In [1]:
%matplotlib inline

In [ ]:
import os
import arcd
import torch
import numpy as np
import matplotlib.pyplot as plt
import openpathsampling as paths

In [3]:
# change to the working directory of choice
# (should be the same as in the first notebook)
wdir = 'SimData_pytorch_toy_22dim'
#wdir = None
if wdir is not None:
    os.chdir(wdir)

### The easy way
By continuing an `openpathsampling` TPS simulation that has been set up with arcd you will autmatically load the `RCModel` saved automagically after the simulation. This will furhtermore repopulate the attached `TrainSet` with the shooting results from the `openpathsampling.Storage`.
To do this, simply continue a TPS simulation the OPS way.

In [4]:
# open old ops storage for appending
storage = paths.Storage('pytorch_toy_22dim.nc', 'a')

In [5]:
# get the PathSampling from storage and set its state to the last MCStep in storage
sampler = storage.pathsimulators[0]
sampler.restart_at_step(storage.steps[-1])

Restoring RCModelSelector without model.Please take care of resetting the model yourself.


In [6]:
# lets get the model and the trainset from the arcd storage
arcd_store = arcd.Storage('arcd_storage.h5', mode='a')
model = arcd_store.rcmodels["most_recent"]  # the last model will alwyas be saved as "most_recent"
# this will restore any ops collective variables used as descriptor transform for the model
model = model.complete_from_ops_storage(storage)
# for the traisnet passing an ops storage automatically resets all ops objects to the values they had at save time
trainset = arcd_store.load_trainset(ops_storage=storage)

In [7]:
# create our hooks
trainhook = arcd.ops.TrainingHook(model, trainset)
densityhook = arcd.ops.DensityCollectionHook(model)
storehook = arcd.ops.ArcdStorageHook(arcd_store, model, trainset)

In [8]:
# and attach them
sampler.attach_hook(trainhook)
sampler.attach_hook(densityhook)
sampler.attach_hook(storehook)

In [9]:
# the only thing left is to take care of the waring above:
# we need to put the model into the RCModel selector 
# (because it can not be saved to ops storages together with the selector)
# if you want to (re)set all RCModels in all RCModelSelectors of a ops simulation to the same model,
# you can use one of the arcd utility functions
# Note: I think this is what you most likely want, since most people will either use only one RCModelSelector or the same Model in all Selectors (?)
arcd.ops.utils.set_rcmodel_in_all_selectors(model=model, simulation=sampler)

In [10]:
# now we can simply run the simulation again and it will start where we left of
sampler.run(100)

Working on Monte Carlo cycle number 600
Running for 3 minutes 33 seconds -  2.16 seconds per step
Estimated time remaining: 2 seconds
DONE! Completed 600 Monte Carlo cycles.


In [11]:
# close the storages
#storage.sync_all()
storage.close()
arcd_store.close()